In [ ]:
%pip install requests ddgs pandas

In [ ]:
from ddgs import DDGS 
import requests
import os
import pandas as pd

In [ ]:
save_folder = "downloaded_images"
os.makedirs(save_folder, exist_ok=True)

def download_images(query, max_results=5):
    success = 0
    with DDGS() as ddgs:
        results = ddgs.images(
            query=query, 
            max_results=max_results
        )

        for i, img in enumerate(results):
            url = img['image']
            
            # Skip webp images by checking URL
            if url.lower().endswith('.webp'):
                print(f'Skipped webp: {url}')
                continue
                
            try:
                response = requests.get(url, timeout=10)
                response.raise_for_status()
                
                # Skip webp images by checking Content-Type
                content_type = response.headers.get('Content-Type', '')
                if 'webp' in content_type.lower():
                    print(f'Skipped webp (content-type): {url}')
                    continue
                
                filename = f'{save_folder}/{query.replace(" ", "_")}_{i+1}.jpg'

                with open(filename, 'wb') as f:
                    f.write(response.content)
                print(f'Downloaded: {filename}')
                success += 1
            except requests.exceptions.RequestException as e:
                print(f'Failed to download {url}: {e}')

    print(f'Total images downloaded for "{query}": {success}/{max_results}')

# download_images("Hanging Bridge in Hagonoy", max_results=5)


Load the CSV file into a DataFrame

- Take only the 'name' and 'municipality' columns
- Turn it into a list of tuples
- Make a function that will return this: f{name} in {municipality}


In [ ]:
df = pd.read_csv('poi.csv')
df = df[['name', 'municipality']]
df_dict = df.to_dict(orient='records')
df_dict

def generate_search_queries(poi_list):
    queries = []
    for poi in poi_list:
        name = poi['name']
        municipality = poi['municipality']
        query = f"{name} in {municipality} Bulacan"
        queries.append(query)
    return queries

search_queries = generate_search_queries(df_dict)

In [ ]:
size = len(search_queries)
size

Let the scraping commence!

In [ ]:

for idx, query in enumerate(search_queries, 1):
    print(f"\n{'='*60}")
    print(f"Progress: {idx}/{size} - Processing: {query}\r")
    print(f"{'='*60}")
    download_images(query, 5)
    clear_output(wait=True)

    
    